In [1]:
#Import all needed files
import csv
import sys, os, re
import time
import json

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.signal import butter, lfilter

import warnings

In [12]:
#Import trial data
file_path = r"C:\Users\jgx7497\Documents\BME 390-3\Jay_Quarter2_Trial(in).csv"
df = pd.read_csv(file_path)

In [46]:
# Read the CSV into a DataFrame.
df = pd.read_csv(file_path)
    
# Define the required columns.
required_cols = ['Time(ms)', 'Acc_x(g)', 'Acc_y(g)', 'Acc_z(g)',
                 'Gyro_x', 'Gyro_y', 'Gyro_z', 'CO2(ppm)']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise ValueError("Missing required columns: " + ", ".join(missing_cols))


In [53]:
#Identify the accel/gyro rows and CO2 rows respectively
co2_rows = df[df['CO2(ppm)'] != 0].copy()
sensor_rows = df[(df[['Acc_x(g)', 'Acc_y(g)', 'Acc_z(g)', 'Gyro_x', 'Gyro_y', 'Gyro_z']] != 0).any(axis=1)].copy()
#Define tolerance in ms
tolerance = 5
merged_rows = []

#Prepare an empty list to collecte the merged rows
for idx, co2_row in co2_rows.iterrows():
    #Find sensor rows within +/- 3 ms
    mask = (sensor_rows["Time(ms)"] >= co2_row["Time(ms)"] - tolerance) & (sensor_rows['Time(ms)'] <= co2_row['Time(ms)'] + tolerance)
    nearby_sensors = sensor_rows[mask]

    if not nearby_sensors.empty:
        best_sensor = nearby_sensors.iloc[(nearby_sensors['Time(ms)'] - co2_row['Time(ms)']).abs().argsort()].iloc[0]
        # Merge the two rows into one
    merged_row = {
        'Time(ms)': co2_row['Time(ms)'],
        'Acc_x(g)': best_sensor['Acc_x(g)'],
        'Acc_y(g)': best_sensor['Acc_y(g)'],
        'Acc_z(g)': best_sensor['Acc_z(g)'],
        'Gyro_x': best_sensor['Gyro_x'],
        'Gyro_y': best_sensor['Gyro_y'],
        'Gyro_z': best_sensor['Gyro_z'],
        'CO2(ppm)': co2_row['CO2(ppm)']
    }
    merged_rows.append(merged_row)

merged_df = pd.DataFrame(merged_rows)
